In [1]:
import os
import numpy as np
import polars as pl
import pandas as pd
import xarray as xr
from datetime import datetime

In [33]:
station_metadata_df = pl.read_csv("../data/cosmo-e.csv", separator=";", n_rows=5, skip_rows=17, has_header=False)

In [34]:
cols = station_metadata_df.columns[1:]

In [35]:
station_codes = station_metadata_df.select(cols).row(0)
longitudes     = station_metadata_df.select(cols).row(1)
latitudes      = station_metadata_df.select(cols).row(2)
grid_idx       = station_metadata_df.select(cols).row(3)
elevations     = station_metadata_df.select(cols).row(4)

In [36]:
station_metadata = pl.DataFrame({
    'station': station_codes,
    'longitude': longitudes,
    'latitude': latitudes,
    'elevation': elevations,
    'grid_idx': grid_idx
})

In [37]:
cols_to_cast = ['longitude', 'latitude', 'elevation', 'grid_idx']

station_metadata = station_metadata.with_columns([
    pl.col(c).cast(pl.Float64, strict=False).alias(c)
    for c in cols_to_cast
])

In [38]:
df = pl.read_csv("../data/cosmo-e.csv", separator=";", skip_rows=23)
df = df.slice(2)

In [39]:
df = df.with_columns([
    pl.col("time").str.strptime(pl.Datetime, "%Y%m%d %H:%M").alias("forecast_reference_time"),
    pl.col("leadtime").str.split(":").list.get(0).cast(pl.Float64).alias("t")
])

In [40]:
base_vars = ['T_2M', 'FF_10M', 'DD_10M', 'TOT_PREC', 'RELHUM_2M', 'DURSUN']

In [41]:
for var in base_vars:
    var_cols = [col for col in df.columns if col == var or (var in col and "_duplicated_" in col)]

    if not var_cols:
        print(f"  Warning: No columns found for {var}")
        continue

    print(f"  {var}: Processing {len(var_cols)} ensemble members")

    # Convert to numeric and handle missing values (-999.0)
    for col in var_cols:
        df = df.with_columns([
            pl.when(pl.col(col).cast(pl.Float64) == -999.0)
            .then(None)
            .otherwise(pl.col(col).cast(pl.Float64))
            .alias(col)
        ])

    # Calculate ensemble average
    df = df.with_columns([
        pl.mean_horizontal([pl.col(c) for c in var_cols])
        .alias(f"coe:{var.lower()}_ensavg")
    ])

  T_2M: Processing 21 ensemble members
  FF_10M: Processing 21 ensemble members
  DD_10M: Processing 21 ensemble members
  TOT_PREC: Processing 21 ensemble members
  RELHUM_2M: Processing 21 ensemble members
  DURSUN: Processing 21 ensemble members


In [42]:
result_cols = ["stn", "forecast_reference_time", "t"] + [f"coe:{var.lower()}_ensavg" for var in base_vars]
result = df.select(result_cols).rename({"stn": "station"})

In [43]:
if "coe:t_2m_ensavg" in result.columns and "coe:air_temperature_ensavg" not in result.columns:
    result = result.with_columns([
        pl.col("coe:t_2m_ensavg").alias("coe:air_temperature_ensavg")
    ])

if "coe:relhum_2m_ensavg" in result.columns and "coe:relative_humidity_ensavg" not in result.columns:
    result = result.with_columns([
        pl.col("coe:relhum_2m_ensavg").alias("coe:relative_humidity_ensavg")
    ])

In [44]:
if "coe:surface_air_pressure_ensavg" not in result.columns:
    result = result.with_columns([
        pl.lit(1013.25).alias("coe:surface_air_pressure_ensavg")
    ])

In [45]:
pd_result = pd.DataFrame(result.to_dict(as_series=True))

In [46]:
def calculate_dewpoint(row):
    T = row["coe:air_temperature_ensavg"]
    RH = row["coe:relative_humidity_ensavg"]

    if pd.isna(T) or pd.isna(RH) or RH <= 0:
        return np.nan

    a = 17.27
    b = 237.7
    alpha = ((a * T) / (b + T)) + np.log(RH/100.0)
    return (b * alpha) / (a - alpha)

In [47]:
pd_result["coe:dew_point_temperature_ensavg"] = pd_result.apply(calculate_dewpoint, axis=1)

In [48]:
pd_result["coe:dew_point_depression_ensavg"] = (
    pd_result["coe:air_temperature_ensavg"] -
    pd_result["coe:dew_point_temperature_ensavg"]
)

In [49]:
def calculate_mixing_ratio(row):
    """Calculate water vapor mixing ratio from dew point and pressure"""
    T_d = row["coe:dew_point_temperature_ensavg"]
    P = row["coe:surface_air_pressure_ensavg"]

    if pd.isna(T_d) or pd.isna(P):
        return np.nan

    a = 17.368
    b = 238.83
    c = 6.107
    e = c * np.exp((a * T_d) / (b + T_d))
    return 622.0 * (e / (P - e))

In [50]:
pd_result["coe:water_vapor_mixing_ratio_ensavg"] = pd_result.apply(calculate_mixing_ratio, axis=1)

In [51]:
pd_result["time:cos_hourofday"] = pd_result["forecast_reference_time"].dt.hour.apply(
    lambda h: np.cos(2 * np.pi * h / 24)
)
pd_result["time:sin_hourofday"] = pd_result["forecast_reference_time"].dt.hour.apply(
    lambda h: np.sin(2 * np.pi * h / 24)
)
pd_result["time:cos_dayofyear"] = pd_result["forecast_reference_time"].dt.dayofyear.apply(
    lambda d: np.cos(2 * np.pi * d / 365)
)
pd_result["time:sin_dayofyear"] = pd_result["forecast_reference_time"].dt.dayofyear.apply(
    lambda d: np.sin(2 * np.pi * d / 365)
)
pd_result["coe:leadtime"] = pd_result["t"]

In [52]:
column_mapping = {}
for col in pd_result.columns:
    if ':' in col:
        new_col = col.replace(':', '_')
        column_mapping[col] = new_col

In [53]:
if column_mapping:
    pd_result = pd_result.rename(columns=column_mapping)
    print("\nRenamed columns to avoid Windows path issues:")
    for old, new in column_mapping.items():
        print(f"  {old} → {new}")


Renamed columns to avoid Windows path issues:
  coe:t_2m_ensavg → coe_t_2m_ensavg
  coe:ff_10m_ensavg → coe_ff_10m_ensavg
  coe:dd_10m_ensavg → coe_dd_10m_ensavg
  coe:tot_prec_ensavg → coe_tot_prec_ensavg
  coe:relhum_2m_ensavg → coe_relhum_2m_ensavg
  coe:dursun_ensavg → coe_dursun_ensavg
  coe:air_temperature_ensavg → coe_air_temperature_ensavg
  coe:relative_humidity_ensavg → coe_relative_humidity_ensavg
  coe:surface_air_pressure_ensavg → coe_surface_air_pressure_ensavg
  coe:dew_point_temperature_ensavg → coe_dew_point_temperature_ensavg
  coe:dew_point_depression_ensavg → coe_dew_point_depression_ensavg
  coe:water_vapor_mixing_ratio_ensavg → coe_water_vapor_mixing_ratio_ensavg
  time:cos_hourofday → time_cos_hourofday
  time:sin_hourofday → time_sin_hourofday
  time:cos_dayofyear → time_cos_dayofyear
  time:sin_dayofyear → time_sin_dayofyear
  coe:leadtime → coe_leadtime


In [54]:
station_metadata = pd.DataFrame(station_metadata.to_dict(as_series=True))
pd_result['station'] = pd_result['station'].astype(str)
station_metadata['station'] = station_metadata['station'].astype(str)

In [55]:
available_stations = set(pd_result['station'].unique())
metadata_stations = set(station_metadata['station'].unique())
print(f"\nFound {len(available_stations)} stations in data and {len(metadata_stations)} stations in metadata")
if not available_stations.issubset(metadata_stations):
    print(f"Warning: {len(available_stations - metadata_stations)} stations in data not found in metadata")


Found 183 stations in data and 184 stations in metadata


In [56]:
merged_result = pd.merge(pd_result, station_metadata, on='station', how='left')
print(f"Merged dataframe has {merged_result.shape[0]} rows and {merged_result.shape[1]} columns")

Merged dataframe has 3843 rows and 24 columns


In [59]:
missing_coords = merged_result[merged_result['longitude'].isna()]['station'].unique()
if len(missing_coords) > 0:
    print(f"Warning: {len(missing_coords)} stations are missing coordinates after merge: {missing_coords}")

In [60]:
if 'model_height_difference' not in merged_result.columns:
    # In the PCPP paper, this is the difference between model terrain height and actual station elevation
    # Since we don't have model terrain height, we'll use a placeholder based on elevation
    # 10% of elevation is a rough approximation
    merged_result['model_height_difference'] = merged_result['elevation'] * 0.1

In [61]:
common_stations = set(station_metadata['station'].unique()) & set(merged_result['station'].unique())
print(f"Number of common stations: {len(common_stations)}")

# Sadece ortak istasyonlarla çalışın
station_coords = station_metadata[station_metadata['station'].isin(common_stations)]
data_vars = merged_result[merged_result['station'].isin(common_stations)]

Number of common stations: 183


In [62]:
station_coords = station_coords[['station', 'longitude', 'latitude', 'elevation', 'grid_idx']]
if 'model_height_difference' not in station_coords.columns:
    station_coords['model_height_difference'] = station_coords['elevation'] * 0.1
station_coords = station_coords.drop_duplicates('station').set_index('station')

In [63]:
data_cols = [col for col in merged_result.columns if col not in ['longitude', 'latitude', 'elevation', 'grid_idx', 'model_height_difference']]
data_vars = merged_result[data_cols]

In [64]:
# Veri değişkenleriyle xarray dataseti oluşturun
ds = data_vars.set_index(['station', 'forecast_reference_time', 't']).to_xarray()

In [65]:
for col in ['longitude', 'latitude', 'elevation', 'grid_idx', 'model_height_difference']:
    if col in station_coords.columns:
        ds.coords[col] = ('station', station_coords[col].values)

In [66]:
if not os.path.exists('data'):
    os.makedirs('data')

ds.to_zarr('data/features.zarr', mode='w')
print("Saved features.zarr")

D:\ML2025\PCPP\.venv\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
D:\ML2025\PCPP\.venv\Lib\site-packages\zarr\core\array.py:3989: UserWarning: The dtype `StringDType()` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(


Saved features.zarr


D:\ML2025\PCPP\.venv\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
D:\ML2025\PCPP\.venv\Lib\site-packages\zarr\api\asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
